#All

#####Import library

In [ ]:
import numpy as np
import pandas as pd
import imageio
import os
from skimage import io
from IPython.display import Image
from PIL import Image
from scipy.signal import correlate2d
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

#####Read file

In [ ]:
df = pd.read_csv(r"30.csv")
print(df)

#####find uniqe sub_id

In [ ]:
# Extract the unique IDs from the 'sub_id' column
unique_ids = df['sub_id'].unique()

# Print the unique IDs
for unique_id in unique_ids:
    print(unique_id)

####how many uniqe id buy package and save data

#####count every user has how many pkg purchase

In [ ]:
# Count the occurrences of each unique ID
id_counts = df['sub_id'].value_counts()

# Print the ID counts
for unique_id, count in id_counts.items():
    print(f"Unique ID: {unique_id}, Count: {count}")

##### plot data

In [ ]:
import os
import pandas as pd
# Plotting the histogram
plt.figure(figsize=(10, 6))
plt.hist(id_counts, bins=100, alpha=0.75, color='blue')

# Customize the plot
plt.title('Count of Unique IDs')
plt.xlabel('Count')
plt.ylabel('Frequency')

# Display the plot
plt.show()

##### create csv file for every unique id

In [ ]:

import os
import shutil
import pandas as pd

#first delete


folder_path = r'H:\GIT project\Hamrah Aval\unique_id_behavior'

try:
    shutil.rmtree(folder_path)
    print(f"Folder '{folder_path}' has been deleted successfully.")
except OSError as e:
    print(f"Error: {e}")
#then create folder
# Create the folder if it doesn't exist
folder_name = 'unique_id_behavior'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Create a dictionary to store lists for each unique ID
id_lists = {unique_id: [] for unique_id in id_counts.index}

# Iterate over each row and append to the corresponding list
for index, row in df.iterrows():
    sub_id = row['sub_id']
    id_lists[sub_id].append(row.tolist())

minimum_data_needed = 5
# Save each list as a separate CSV file in the folder
for unique_id, rows_list in id_lists.items():
    if len(rows_list) >= minimum_data_needed:
        # Create a DataFrame from the list of rows
        sub_df = pd.DataFrame(rows_list, columns=df.columns)

        # Sort the DataFrame based on the "actvn_dt" column
        sub_df = sub_df.sort_values(by='actvn_dt')

        # Add a new column "times" with incrementing values starting from 1
        sub_df['times'] = range(1, len(sub_df) + 1)

        # Save the DataFrame as a CSV file in the folder
        file_path = os.path.join(folder_name, f'{unique_id}_30.csv')
        sub_df.to_csv(file_path, index=False)
        # print(f'Saved rows for Unique ID {unique_id} in {file_path}')
    else:
        print(f'Skipped saving rows for Unique ID {unique_id} (less than {minimum_data_needed} rows)')

##### make it zip file

In [ ]:
import zipfile
# Create a zip file of the folder
zip_file_name = 'unique_id_behavior.zip'
with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(folder_name):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, os.path.basename(file_path))

print(f'Successfully created {zip_file_name}.')

#### train and predict last bought

###### nom to word and inverse

In [ ]:
from word2number import w2n
import inflect

def words_to_number(words):
    try:
        number = w2n.word_to_num(words)
        return number
    except ValueError as e:
        print(f"Error: {e}")
        return None
    
print(words_to_number("thirty-nine"))

def number_to_words(number):
    p = inflect.engine()
    return p.number_to_words(number)


##### predict next pkg linear reg

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
# Function to predict the next package using Linear Regression
def predict_next_pkg_linear(X_train, y_train, next_time):
    # Feature scaling using StandardScaler
    sc_X = StandardScaler()
    X_train_scaled = sc_X.fit_transform(X_train)

    # Encode the labels with OneHotEncoder
    encoder = OneHotEncoder(sparse=False)
    encoded_features = encoder.fit_transform(y_train)
    # print(X_train_scaled)
    # Fit Linear Regression model
    lin_reg = LinearRegression()
    lin_reg.fit(X_train_scaled, encoded_features)

    # Perform decoding
    next_time_scaled = sc_X.transform(np.array([next_time]))
    predicted_features = lin_reg.predict(next_time_scaled)
    
    # Inverse transform the one-hot encoded matrix to get the predicted label
    decoded_labels = encoder.inverse_transform(predicted_features)

    return decoded_labels


##### predict next pkg SVM reg

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn import svm

# Function to predict the next package using Linear Regression
def predict_next_pkg_svm_(X_train, y_train, next_time):
    # Feature scaling using StandardScaler
    sc_X = StandardScaler()
    X_train_scaled = sc_X.fit_transform(X_train)
    # print(y_train)

    unique_values_count = len(set(y_train))
    unique_values_count = len({x: True for x in y_train})
    if(unique_values_count != 1):
        classifier = svm.SVC( kernel = 'linear', decision_function_shape='ovr')  # You can choose different kernels like 'linear', 'rbf', 'poly', etc.
        # Train the model
        classifier.fit(X_train_scaled, y_train)
        # Perform decoding
        next_time_scaled = sc_X.transform(np.array([next_time]))
        predicted_features = classifier.predict(next_time_scaled)
        predicted_pkg =words_to_number(str(predicted_features[0]))
        
        return predicted_pkg
    else:
        return words_to_number(y_train[-1])
    # return decoded_labels


##### predict next with naive_bayes 

In [ ]:
from sklearn.naive_bayes import GaussianNB


# Function to predict the next package using Linear Regression
def predict_next_pkg_naive_bayess(X_train, y_train, next_time):
    # Feature scaling using StandardScaler
    sc_X = StandardScaler()
    X_train_scaled = sc_X.fit_transform(X_train)
    # print(y_train)

    unique_values_count = len(set(y_train))
    unique_values_count = len({x: True for x in y_train})
    if(unique_values_count != 1):
        classifier = GaussianNB()
        # Train the model
        classifier.fit(X_train_scaled, y_train)
        # Perform decoding
        next_time_scaled = sc_X.transform(np.array([next_time]))
        predicted_features = classifier.predict(next_time_scaled)
        predicted_pkg =words_to_number(str(predicted_features[0]))
        
        return predicted_pkg
    else:
        return words_to_number(y_train[-1])
    # return decoded_labels


##### Read files

In [ ]:
import warnings

# Ignore all warnings
warnings.simplefilter("ignore")

csv_files = [file for file in os.listdir(folder_name) if file.endswith('.csv')]
real_data = []
predict_nex_pkg_data_linear = []
predict_nex_pkg_data_svm = []
predict_nex_pkg_data_naive_bayes = []

# Read each CSV file one by one
columns_ = [5,24]

for file in csv_files:
    # Construct the file path
    file_path = os.path.join(folder_name, file)

    # Read the CSV file
    df = pd.read_csv(file_path)

    # Get information
    X_train = df.iloc[:, columns_].values
    # print(X_train)
    y_train = df["voice_pkg_code"].values.reshape(-1, 1)
    df[['prod_stat_eng', 'gender', 'Pk_ATL/BTL', 'device_type', 'Pk_regular/Event']] = df[['prod_stat_eng', 'gender', 'Pk_ATL/BTL', 'device_type', 'Pk_regular/Event']].apply(LabelEncoder().fit_transform)
    X_train = X_train[:-1]
    y_train = np.delete(y_train, -1).tolist()
    y_train_word = []
    for i in range(len(y_train)):
       y_train_word.append(number_to_words(y_train[i]))
    sub_id = df["sub_id"].iloc[-1]
    next_time = df.iloc[-1, columns_].values
 
    last_pkg_bought = df["voice_pkg_code"].iloc[-1]
    real_data.append( df["voice_pkg_code"].iloc[-1])

    # Predict
    # y_pred_linear = predict_next_pkg_linear(X_train,y_train.reshape(-1, 1) , next_time)
    # predict_nex_pkg_data_linear.append(int(y_pred_linear[0][0]))
    y_pred_svm = predict_next_pkg_svm_(X_train,y_train_word , next_time)
    predict_nex_pkg_data_svm.append(y_pred_svm)
    y_pred_naive_bayes = predict_next_pkg_naive_bayess(X_train,y_train_word , next_time)
    predict_nex_pkg_data_naive_bayes.append(y_pred_naive_bayes)
    # if(last_pkg_bought != int(decoded_labels[0][0])):
      # print(f'{sub_id}  last pkg bought : {last_pkg_bought}  next pkg predicted {int(decoded_labels[0][0])}.')
 

##### accuracy

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score,precision_score, recall_score
print("accuracy_score = ",accuracy_score(real_data,y_pred_naive_bayes))
print(confusion_matrix(real_data, y_pred_naive_bayes))

In [ ]:
# Plot the histograms
plt.figure(figsize=(10, 5))

plt.hist(predict_nex_pkg_data, bins=160, color='blue', alpha=0.5, label='predict')
plt.hist(real_data, bins=160, color='red', alpha=0.5, label='last bought')

plt.xlabel('voice_pkg_code')
plt.ylabel('Frequency')
plt.title('predict with linear regresion and ')
plt.legend()

plt.tight_layout()
plt.show()



#### train and predict every user with times

#####predict next pkg linear reg

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

def predict_next_pkg_linear(X_train,y_train,next_time):
    encoder = OneHotEncoder()

    # Fit the encoder to the 'voice_pkg_code' column
    encoder.fit(y_train.reshape(-1, 1))

    # Transform the 'voice_pkg_code' column into one-hot encoded features
    encoded_features = encoder.transform(y_train.reshape(-1, 1)).toarray()

    # Scale the X_train data
    sc_X = StandardScaler()
    X_train_scaled = sc_X.fit_transform(X_train.reshape(-1, 1))

    lin_reg = LinearRegression()
    lin_reg.fit(X_train_scaled, encoded_features)

    # Perform decoding
    predicted_features = lin_reg.predict(sc_X.transform(np.array([next_time]).reshape(-1, 1)))
    decoded_labels = encoder.inverse_transform(predicted_features)
    return decoded_labels




#####Read files

In [ ]:
# Get a list of all CSV files in the folder
csv_files = [file for file in os.listdir(folder_name) if file.endswith('.csv')]
real_data = []
predict_nex_pkg_data = []
# Read each CSV file one by one
for file in csv_files:
    # Construct the file path
    file_path = os.path.join(folder_name, file)

    # Read the CSV file
    df = pd.read_csv(file_path)

    # Get information
    X_train = np.ravel(np.array(df["times"].values).reshape(-1, 1))
    # X_train = np.ravel(np.array(df["actvn_dt"].values).reshape(-1, 1))
    y_train = np.ravel(np.array(df["voice_pkg_code"].values).reshape(-1, 1))
    sub_id = df["sub_id"].iloc[-1]
    next_time = df["times"].iloc[-1] + 1
    last_pkg_bought = df["voice_pkg_code"].iloc[-1]
    real_data.append( df["voice_pkg_code"].iloc[-1])

    # Predict
    decoded_labels = predict_next_pkg_linear(X_train,y_train , next_time)
    if(last_pkg_bought != int(decoded_labels[0][0])):
      print(f'{sub_id}  last pkg bought : {last_pkg_bought}  next pkg predicted {decoded_labels}.')
    predict_nex_pkg_data.append(int(decoded_labels[0][0]))


In [ ]:
# Plot the histograms
plt.figure(figsize=(10, 5))

plt.hist(predict_nex_pkg_data, bins=160, color='blue', alpha=0.5, label='predict')
plt.hist(real_data, bins=160, color='red', alpha=0.5, label='last bought')

plt.xlabel('voice_pkg_code')
plt.ylabel('Frequency')
plt.title('predict with linear regresion and ')
plt.legend()

plt.tight_layout()
plt.show()

